In [1]:
import os
from pathlib import Path
import re

FOLDER = Path(os.path.dirname(os.path.realpath("__file__"))) / 'data'
in_file = 'day15.txt'

## Part One

In [2]:
def distance(p1,p2):
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

def range_at_row(p1, p2, y):
    d = distance(p1,p2)
    # absolute vertical offset
    y_dist = abs(p1[1] - y)
    
    # the the vertical offset is greater
    # than the manhattan distance, the sensor doesn't reach
    if (y_dist) > d:
        return set()
    
    # absolute horizontal offset is the manhattan dist - y offset
    # so y_dist + h_dist == d
    h_dist = abs(d - y_dist)
    
    return set(range(p1[0] - h_dist, p1[0] + h_dist+1))

sensors = {}
beacons = set()
with open(FOLDER/in_file) as f:
    for line in f:
        s_x, s_y, b_x, b_y = map(int, re.findall(r'[0-9-]+', line))
        sensors[(s_x, s_y)] = b_x, b_y
        beacons.add((b_x, b_y))

row = 2000000
covered = set()
for k, v in sensors.items():
    covered = covered | range_at_row(k, v, row)
len(set(covered)) - 1


5083287

## Part Two

In [3]:
from collections import namedtuple


class Point(namedtuple("Point", ('x', 'y'))):
    def distance(self, other):
        '''Manhattan distance'''
        return abs(self.x - other.x) + abs(self.y - other.y)

    def midpoint(self, other):
        '''Centerpoint of square defined by corners'''
        return Point((self.x + other.x) // 2, (self.y + other.y) // 2)
    
    def clamp_subtract(self, other, minimum=0):
        '''Stay above minimum element-wis'''
        return Point(max(self.x - other.x, minimum), max(self.y - other.y, minimum))
    
    def __sub__(self, other):
        return Point(self.x - other.x, self.y - other.y)

    
with open(FOLDER/in_file) as f:
    sensors = []
    beacons = []
    for line in f:
        sx, sy, bx, by = map(int, re.findall(r'([0-9-]+)', line))
        sensors.append(Point(sx, sy))
        beacons.append(Point(bx, by))

    
def partition_points(tl, mid, br):
    yield from (
        (tl, mid),
        (Point(tl.x, mid.y+1), Point(mid.x, br.y)),
        (Point(mid.x+1, tl.y), Point(br.x, mid.y)),
        (Point(mid.x+1, mid.y+1), br)
    )

def partition(sensors, beacons, side=4000000):
    '''
    Partitions into progressively more restictive quadrants and eliminate
    based on distances.
    '''
    
    distances = [p1.distance(p2) for p1, p2 in zip(sensors, beacons)]
    
    top_left, bottom_right = Point(0, 0), Point(side, side)
    
    stack = [(top_left, bottom_right)]

    while True:
        top_left, bottom_right = stack.pop()
        
        # Finished
        if top_left == bottom_right:
            print(top_left)
            return (top_left.x * side) + top_left.y
        
        middle = top_left.midpoint(bottom_right) 
    
        for top_left, bottom_right in partition_points(top_left, middle, bottom_right):  
            tl_dist = [s.clamp_subtract(top_left) for s in sensors]
            br_dist = [bottom_right.clamp_subtract(s) for s in sensors]

            quad_dist = [sum(t) + sum(b) for t, b in zip(tl_dist, br_dist)]
            
            if all(d1 < d2 for d1, d2 in zip(distances, quad_dist)):
                stack.append((top_left, bottom_right))

partition(sensors, beacons)

Point(x=3283509, y=3205729)


13134039205729